In [1]:
import numpy as np
import pandas as pd
import pickle

## S&P 500 

In [2]:
#import listing of S&P 500 companies
constituents = pd.read_csv("/Users/dunleavyjason/Documents/Metis/covid_disclosures/constituents.csv")

In [3]:
constituents.rename(columns={"Symbol":"ticker", "Name":"name", "Sector":"sector"}, inplace=True)
constituents

,ticker,name,sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,ABIOMED Inc,Health Care
...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [4]:
#import mapping of tickers to SEC ciks
ciks = pd.read_csv("https://www.sec.gov/include/ticker.txt", header=None, names=["data"])

In [5]:
ciks = ciks.data.str.split("\t", expand=True)
ciks.rename(columns={0:"ticker", 1:"cik"}, inplace=True)
ciks["ticker"] = ciks.ticker.str.upper()
ciks

,ticker,cik
0,AAPL,320193
1,MSFT,789019
2,AMZN,1018724
3,GOOG,1652044
4,TCEHY,1293451
...,...,...
11185,WTID,1114446
11186,WTIU,1114446
11187,WTREP,1601669
11188,XDIV,1688487


In [6]:
sp_500 = (constituents.set_index("ticker").join(ciks.set_index("ticker"), how="inner")).reset_index()
sp_500

,ticker,name,sector,cik
0,A,Agilent Technologies Inc,Health Care,1090872
1,AAL,American Airlines Group,Industrials,6201
2,AAP,Advance Auto Parts,Consumer Discretionary,1158449
3,AAPL,Apple Inc.,Information Technology,320193
4,ABBV,AbbVie Inc.,Health Care,1551152
...,...,...,...,...
499,YUM,Yum! Brands Inc,Consumer Discretionary,1041061
500,ZBH,Zimmer Biomet,Health Care,1136869
501,ZBRA,Zebra Technologies,Information Technology,877212
502,ZION,Zions Bancorp,Financials,109380


In [8]:
sp_500_series = pd.Series(sp_500["cik"].values, index=sp_500["ticker"])
sp_500_series

ticker
A       1090872
AAL        6201
AAP     1158449
AAPL     320193
ABBV    1551152
         ...   
YUM     1041061
ZBH     1136869
ZBRA     877212
ZION     109380
ZTS     1555280
Length: 504, dtype: object

In [9]:
#create dictionary to use in API below
sp_500_dict = sp_500_series.to_dict()

In [10]:
#view sample
sample_dict = sp_500_series.iloc[0:3].to_dict()
sample_dict

{'A': '1090872', 'AAL': '6201', 'AAP': '1158449'}

## SEC API - Obtain Financial Statement Filing Info

In [11]:
# package used to execute HTTP POST request to the API
import json
import urllib.request
import time, os
import random

In [12]:
# API endpoint
TOKEN = "4576f2138046df6746527c9675d862c2658ddb99d1c7db6ff99b728c6faf1ebe"
API = "https://api.sec-api.io?token=" + TOKEN

In [15]:
qa_list = []

In [16]:
for cik in sp_500_dict.values():
    
    payload = {
      "query": { "query_string": { "query":"cik:{}".format(cik) + " AND filedAt:{2020-01-01 TO 2021-02-23} AND (formType:\"10-Q\" OR formType:\"10-K\")"}},
      "from": "0",
      "size": "100",
      "sort": [{ "filedAt": { "order": "desc" } }]
    }

    jsondata = json.dumps(payload) # format your payload to JSON bytes
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes

    req = urllib.request.Request(API) # instantiate the request 

    req.add_header('Content-Type', 'application/json; charset=utf-8') # set the correct HTTP header: Content-Type = application/json
    req.add_header('Content-Length', len(jsondataasbytes)) # set the correct length of your request

    response = urllib.request.urlopen(req, jsondataasbytes) # send the request to the API

    res_body = response.read() # read the response 
    filings = json.loads(res_body.decode("utf-8")) # transform the response into JSON

    filing_dict = filings["filings"]
    
    for ident in filing_dict:
        qa = {}
        qa["cik"] = ident["cik"]
        qa["company_name"] = ident["companyName"]
        qa["filed_at"] = ident["filedAt"][:10]
        qa["form_type"] = ident["formType"]
        qa["linkToTxt"] = ident["linkToTxt"]
        qa_list.append(qa)

In [17]:
urls_df = pd.DataFrame(qa_list)
urls_df

,cik,company_name,filed_at,form_type,linkToTxt
0,1090872,"AGILENT TECHNOLOGIES, INC.",2020-12-17,10-K,https://www.sec.gov/Archives/edgar/data/109087...
1,1090872,"AGILENT TECHNOLOGIES, INC.",2020-09-01,10-Q,https://www.sec.gov/Archives/edgar/data/109087...
2,1090872,"AGILENT TECHNOLOGIES, INC.",2020-05-29,10-Q,https://www.sec.gov/Archives/edgar/data/109087...
3,1090872,"AGILENT TECHNOLOGIES, INC.",2020-03-03,10-Q,https://www.sec.gov/Archives/edgar/data/109087...
4,6201,American Airlines Group Inc.,2021-02-17,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...
...,...,...,...,...,...
2332,1555280,Zoetis Inc.,2021-02-16,10-K,https://www.sec.gov/Archives/edgar/data/155528...
2333,1555280,Zoetis Inc.,2020-11-05,10-Q,https://www.sec.gov/Archives/edgar/data/155528...
2334,1555280,Zoetis Inc.,2020-08-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...
2335,1555280,Zoetis Inc.,2020-05-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...


In [18]:
urls_df.drop_duplicates(inplace=True)
urls_df

,cik,company_name,filed_at,form_type,linkToTxt
0,1090872,"AGILENT TECHNOLOGIES, INC.",2020-12-17,10-K,https://www.sec.gov/Archives/edgar/data/109087...
1,1090872,"AGILENT TECHNOLOGIES, INC.",2020-09-01,10-Q,https://www.sec.gov/Archives/edgar/data/109087...
2,1090872,"AGILENT TECHNOLOGIES, INC.",2020-05-29,10-Q,https://www.sec.gov/Archives/edgar/data/109087...
3,1090872,"AGILENT TECHNOLOGIES, INC.",2020-03-03,10-Q,https://www.sec.gov/Archives/edgar/data/109087...
4,6201,American Airlines Group Inc.,2021-02-17,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...
...,...,...,...,...,...
2332,1555280,Zoetis Inc.,2021-02-16,10-K,https://www.sec.gov/Archives/edgar/data/155528...
2333,1555280,Zoetis Inc.,2020-11-05,10-Q,https://www.sec.gov/Archives/edgar/data/155528...
2334,1555280,Zoetis Inc.,2020-08-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...
2335,1555280,Zoetis Inc.,2020-05-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...


In [19]:
urls_df["filed_at"] = pd.to_datetime(urls_df.filed_at)

In [20]:
urls_df[urls_df["linkToTxt"].isnull()] #check for null urls

,cik,company_name,filed_at,form_type,linkToTxt


In [21]:
urls = pd.merge(urls_df, sp_500, how="left", on="cik", )
urls

,cik,company_name,filed_at,form_type,linkToTxt,ticker,name,sector
0,1090872,"AGILENT TECHNOLOGIES, INC.",2020-12-17,10-K,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
1,1090872,"AGILENT TECHNOLOGIES, INC.",2020-09-01,10-Q,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
2,1090872,"AGILENT TECHNOLOGIES, INC.",2020-05-29,10-Q,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
3,1090872,"AGILENT TECHNOLOGIES, INC.",2020-03-03,10-Q,https://www.sec.gov/Archives/edgar/data/109087...,A,Agilent Technologies Inc,Health Care
4,6201,American Airlines Group Inc.,2021-02-17,10-K,https://www.sec.gov/Archives/edgar/data/6201/0...,AAL,American Airlines Group,Industrials
...,...,...,...,...,...,...,...,...
2334,1555280,Zoetis Inc.,2021-02-16,10-K,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care
2335,1555280,Zoetis Inc.,2020-11-05,10-Q,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care
2336,1555280,Zoetis Inc.,2020-08-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care
2337,1555280,Zoetis Inc.,2020-05-06,10-Q,https://www.sec.gov/Archives/edgar/data/155528...,ZTS,Zoetis,Health Care


In [22]:
#export final dataframe
urls.to_pickle("urls.pkl")